In [ ]:
# The code was removed by Watson Studio for sharing.

## The Battle of Neighborhoods (Week 1)
### Coursera Capstone Project
### Part 1 : Introduction and Data Sections

### I.   Introduction 
### Problem background:
   The Covid-19 Pandemic started from Wu Hang, China at the end of 2019 and is spreading all around the world. 
Until 06/24/2020, the pandemic affected 9,392,847 cases, with 2% is serious and 480,559 deaths or 9% from the closed cases (1). The United States is the leading country damaged by this virus. Following CDC, there are more than 2.3 million people contracting this virus with more than 120 thousand people died (2).
In that tragedy, New York city is the epicenter of the USA with more than 200 000 cases and 17 600 deaths (3). High density of population, very quick transmission from person to person and the lack of protect medical equipment such as masks, protective cloths and ventilators play the key role to push the big apple to the chaos.
With the effort and sacrifice of medical team and the New York leaders, the pandemic is temporally controlled, and New Yorker stepped to the second phase of re-opening in June 22nd 2020 (3).

   Re-opening the business is important, but it is high risk of second wave of pandemic outbreak. That is a big challenge for both government and other organizations to help the residents go back to work with the social distancing and face masks, but there are not available cures and vaccines. Mental and material help is never enough in this period. How can an organization or an individual support the New York city resident efficiently in this situation? A reasonable strategy and good understanding the pandemic impact on NYC are important steps for successful support service.
 
    
### Problem description:

   A non-profit organization has a project to support the New York City (NYC) hospitals. The organization manager wants to know where the good place to open office and storage to supply N95 masks and hand sanitizer for all NYC hospitals in the most impacted borough by the Covid 19 virus. From this model, the organization will develop in other boroughs or cities if needed. In order to get the right plan, there are 2 questions need to be resolved:
 1.	What is the most affected borough by the virus?
 2.	where is the reasonable place to open office or storage closed to all hospitals in that borough?
    
    
   To resolve the problems, various factors need to be considered and analyzed to figured out the right solution such as:
    
 * NYC population and demographics 
 * The Covid 19 virus affected cases, hospitalized cases and death until re-opening phase in June 22nd, 2020. 
 * List of Hospitals in NYC. And the other data related need to be analyzed. 


### Interested Audience

The objective is to evaluate the most affected borough and to locate the good location in NYC closed to hospitals. This project is applicable to all profit and non-profit organizations who are interested in medical supply or even individual who want to do business in medical field.  

### Reference
1. https://www.worldometers.info/coronavirus/
2. https://www.cdc.gov/coronavirus/2019-ncov/cases-updates/us-cases-deaths.html   
3. https://www1.nyc.gov/site/doh/covid/covid-19-data.page
4. https://www.mapsofworld.com/usa/states/new-york/nyc-boroughs-map.html